# Analysis on Stairway working hours

Based on Google Calendar data.

Exported to ICS file, then free converted to CSV using https://www.projectwizards.net/en/support/ics2csv-converter

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import and prep data

In [ ]:
!ls '../../data/google-calendar'

In [ ]:
column_names = ['title', 'start_time', 'end_time', 'notes', 'assigned_resources', 'additional_title']

df = pd.read_csv('../../data/google-calendar/stairway-to-travel-calendar.csv', 
                 sep="\t", names=column_names, header=0, parse_dates=[1, 2])

# create some usefull features
df['work_time'] = df['end_time'] - df['start_time']
df['work_hours'] = df['work_time'] / np.timedelta64(1, 'h')
df["week_date"] = (df['start_time'] - df['start_time'].dt.weekday * datetime.timedelta(days=1)).dt.date

In [ ]:
df.head()

### Simple stats

Period of administration

In [ ]:
df['start_time'].min(), df['start_time'].max()

In [ ]:
df['start_time'].max() - df['start_time'].min()

Total time invested in this period

In [ ]:
df['work_hours'].sum()

## Plot weekly working hours over time

Create an index to add missing weeks. Set working hours to zero for missing weeks.

In [ ]:
all_week_dates = pd.date_range(df["week_date"].min(), df["week_date"].max(), freq='W-MON')

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.gca()

hours_per_week = (
    df
    .groupby('week_date')
    .agg({
        'work_hours' : sum
    })
    .reindex(all_week_dates, fill_value=0)
    .assign(rolling_mean = lambda df: df.rolling(window=4).mean())
    .assign(overall_mean = lambda df: df['work_hours'].mean())
)

hours_per_week.plot(ax=ax)
ax.set_title('Weekly working hours on Stairway to Travel', fontsize=16)
ax.set_ylabel('Worked hours per week');

Conclusie: Los van vakanties en met name onze bruiloft, gemiddeld >1 dag per week.

## Analyse time spent on

**TODO**: Find out how much time I spent on front-end, back-end or business work by extracting labels from text.

## Analyse when during the week I typically work

Make heatmap with hours of day on y-axis (00.00-24.00) and day of week on x-axis (Mon-Sun). Then light up the hours that I have worked.

Start by picking one record that has an odd start/end time that doesn't perfectly align with a `15min`, `30min`, or `h` interval. Use this to develop the code.

In [ ]:
row = df.iloc[189]
# row

Below code is partially copied from [this example](https://stackoverflow.com/questions/43154462/python-pandas-transform-start-and-end-datetime-range-stored-as-2-columns-to-i), but adjusted to make it work for our dataframe.

In [ ]:
def expand_datetime_range(row, freq = '15min'):
    intervals = pd.date_range(row["start_time"].floor(freq), row["end_time"].ceil(freq), freq=freq)
    df = (
        pd.DataFrame({'start_interval': intervals[:-1], 'end_interval' : intervals[1:]})
        .assign(start_time = row['start_time'], end_time = row['end_time'], title = row['title'])
    )
    return df

def apply_function_to_full_df(df, function, freq = '15min'):
    dfs = pd.concat([function(series, freq) for idx, series in df.iterrows()])
    return dfs

Try it out on the example record, to see if it works:

In [ ]:
(
    expand_datetime_range(row, 'h')
    .assign(overlap_time = lambda row: row[['end_interval', 'end_time']].min(axis=1) - row[['start_interval', 'start_time']].max(axis=1))
)

Great! So now apply it to the entire df:

In [ ]:
freq = 'h'

df_intervals = (
    apply_function_to_full_df(df, expand_datetime_range, freq=freq)
    # calculate intervals
    .assign(overlap_time = lambda row: row[['end_interval', 'end_time']].min(axis=1) - row[['start_interval', 'start_time']].max(axis=1))
    .assign(overlap = lambda row: row['overlap_time'] / (row['end_interval'] - row['start_interval']))
    # set time variables for heatmap
    .assign(time = lambda row: row['start_interval'].dt.time)
    .assign(weekday = lambda row: row['start_interval'].dt.weekday)
#     .assign(weekday = lambda row: row['start_interval'].dt.strftime("%A"))
)

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.gca()

# pivot data for heatmap
df_heatmap = (
    df_intervals
    .pivot_table(index='time', columns='weekday', values='overlap', aggfunc='sum')
)

ax = sns.heatmap(df_heatmap)
ax.set_title('Time in the week that I work on Stairway to Travel', fontsize=16);

Interesting, according to the data, I work most when:
* In the evening hours between 20.00 and 22.00, except for Friday and Saturday nights (party!)
* During daytime on Wednesdays or Fridays. These are respectively my free day and GoDataDriven Friday
* Sundays in the afternoon!

Which I indeed recognize.

There is one odd night in which I pulled an all nighter. Looking into this event:

In [ ]:
df_intervals.loc[lambda df : df['time'] == datetime.time(3, 0)]

I remember: this was the flight back from San Fransisco! 

Done.